## Práctica 02, Reporte 01
## Daniel Ramírez Umaña
## 13 Septiembre 2021

## How to Remove Outliers for Machine Learning

Limpiamos nuestros conjuntos con la finalidad de que estos sean lo más representativos posible del problema.

En muchos casos los conjuntos tienen valores que se alejan mucho de los demás datos y que se comportan de forma extraña. A estos los llamamos **valores atípicos** o bien ***outliers***.

### 1. Qué es un Valor Atípico (outlier)

Este es una observación que de una forma u otra no se parece a las demás o bien que no encaja de alguna manera.

Algunas de las causas de un valor atípico son:
* Error de medición o de entrada.
* Corrupción de datos.
* Observación atípica real.

Si bien no hay forma precisa para detectar *outliers*, es posible utilizar métodos estadísticos para identificar observaciones que parecieran serlas en relación a los datos disponibles.

### 2. Conjunto de Datos de Prueba

In [1]:
# generate gaussian data
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# summarize
print('mean=%.3f stdv=%.3f' % (mean(data), std(data)))

mean=50.049 stdv=4.994


### 3. Método de la Desviación Estándar

Es posible usar la desviación estándar de la muestra como límite para identificar los valores atípicos, si sabemos que la distribución de los valores de la muestra es Gaussiana o similar a la Gaussiana.

Podemos calcular la media y la desviación estándar de una muestra dada, y luego calcular el punto de corte para identificar los valores atípicos como más de 3 desviaciones estándar de la media.

In [4]:
# identify outliers with standard deviation
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# calculate summary statistics
data_mean, data_std = mean(data), std(data)
# identify outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off

# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))

# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Identified outliers: 29
Non-outlier observations: 9971


Como podemos observar, 29 valores correspondientes a muy posibles *outliers* han sido eliminados

### 4. Método del Rango Intercuartil

Aveces no es recomendable extraer valores tratándolos con una distribución gaussiana, como lo es el caso en que los datos no son normales o no lo suficientemente normales. Para estos casos podemos usar el **Rango Intercuartil** (IQR) el cual corresponde a la diferencia entre los percentiles 25 y 75 de los datos y se dedine la caja en un gráfico de cajas y bigote (box in a box and whisker plot).

In [5]:
# identify outliers with interquartile range
from numpy.random import seed
from numpy.random import randn
from numpy import percentile
# seed the random number generator
seed(1)
# generate univariate observations
data = 5 * randn(10000) + 50
# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in data if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))

Percentiles: 25th=46.685, 75th=53.359, IQR=6.674
Identified outliers: 81
Non-outlier observations: 9919


Podemos ver que para este conjunto y dado el IQR fueron eliminadas 81 observasiones.

### 5. Detección Automática de Valores Atípicos

En **aprendizaje automático** o ***Machine Learning***, es posible realizar la clasificación de una clase para la detección de valores atípicos. A esto se le conoce como la **Clasificación de una clase** o ***One-Class Classification*** (OCC) que consiste en ajustar un modelo de datos "normales" y predecir de los nuevos datos son normales o un valor atípico.

En casos de variables de baja dimensionalidad esta técnica podría funcionar, pero su fiabilidad se ve reducida con relación al aumento de características (a esto se le conoce como **la maldición de ls dimensionalidad**.

In [6]:
# load and summarize the dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the dataset
print(X.shape, y.shape)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(506, 13) (506,)
(339, 13) (167, 13) (339,) (167,)


In [7]:
# evaluate model on the raw dataset
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 3.417


In [8]:
# evaluate model on training dataset with outliers removed
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = read_csv(url, header=None)
# retrieve the array
data = df.values
# split into inpiut and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(339, 13) (339,)
(305, 13) (305,)
MAE: 3.356


Podemos ver el cambio en MAE una vez eliminados los *outliers*.

### Lo aprendido y el posible uso en mi futuro profesional:

Aprendí que los valores atípicos pueden jugarme una mala pasada a la hora de entrenar mis modelos o bien a la hora de sacar conclusiones con respecto a mis conjuntos.

En mi futuro profesional como Científico de Datos, será necesario tener en cuenta que debo chequear de la exxistancia de datos atípicos y tomar deseciones de qué hacer con ellos para mejorar mis modelos.

### Aspectos de los que me gustaría saber más o que no me quedaron muy claros:

Ya antes he trabajado con *outliers* en es sacar conclusiones con respecto a valores estadísticos, pero me gustaría ahora aplicarlo a un proyecto antes de utilizar un algoritmo clasificador.